In [43]:
import pandas as pd
import openai
from pprint import pprint 
from collections import Counter
import json
import os
from dotenv import load_dotenv

load_dotenv('openai.env')
openai.api_key  = os.getenv('OPENAI_API_KEY')

# https://github.com/swar/nba_api/blob/master/docs/examples/PlayByPlay.ipynb

True

# utils

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.8, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

with open('resource/EVENTMSGTYPE_glossary.json') as f:
    EVENTMSGTYPE = json.load(f)
EVENTMSGTYPE = {int(key): value for key, value in EVENTMSGTYPE.items()}

# prepare data

In [52]:
df = pd.read_csv('resource/playbyplayv2_0012000001.csv') # 'EVENTMSGACTIONTYPE'

dfp1, dfp2, dfp3 = df[['PLAYER1_NAME', 'EVENTMSGTYPE']].rename(columns={'PLAYER1_NAME':'PLAYER'}), \
                    df[['PLAYER2_NAME', 'EVENTMSGTYPE']].rename(columns={'PLAYER2_NAME':'PLAYER'}), \
                    df[['PLAYER3_NAME', 'EVENTMSGTYPE']].rename(columns={'PLAYER3_NAME':'PLAYER'})
                    
df_event = pd.concat([dfp1, dfp2, dfp3], axis=0)
df_event = df_event.loc[df_event['PLAYER'].notnull()].\
                    groupby('PLAYER').agg(lambda *x: x).map(lambda x: Counter(x[0]))
                    
df_event_count = pd.DataFrame.from_records(df_event['EVENTMSGTYPE'],index=df_event.index).\
                              fillna(0).astype(int).rename(columns=EVENTMSGTYPE)
                              
event_count_for_prompt =   df_event_count.to_csv()    
df_event_count                 


C:\Users\user\AppData\Local\Temp\ipykernel_10080\1215812201.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_event_count = pd.DataFrame.from_records(df_event['EVENTMSGTYPE'],index=df_event.index).\


,Field Goal Made,Field Goal Missed,Rebound,Free Throw Attempt,Substitution,Foul,Turnover,Jump Ball,Violation
PLAYER,,,,,,,,,
Aaron Gordon,10,3,3,3,2,2,0,0,0
Bogdan Bogdanovic,5,7,4,2,5,1,2,0,0
Brandon Goodwin,6,7,2,0,7,1,2,0,0
Bruno Fernando,2,5,9,4,4,7,0,1,0
Cam Reddish,7,6,8,4,7,4,7,0,0
Chuma Okeke,4,6,3,2,1,4,1,0,0
Clint Capela,4,2,14,1,5,3,2,1,0
Cole Anthony,6,7,1,0,4,6,4,0,0
Danilo Gallinari,3,5,4,8,6,9,1,0,0


# create a report


In [67]:
size_report = 800

prompt = f""" 
You’re a first-class sports basketball columnist.\
    You find interesting and unusual moments of the match and tell about them.
You are given the data:
{event_count_for_prompt}
You have to:
1) Find interesting information about events and players
2) Combine them into a fascinating little emotional post
3) You should only talk about the best and worst players
4) Your answer should be limited to {size_report} characters

"""  

res = get_completion(prompt, model="gpt-3.5-turbo")
pprint(res)

('In a thrilling basketball match, there were some interesting and unusual '
 "moments that caught everyone's attention. Let's dive into the highlights!\n"
 '\n'
 'Among the best players, Aaron Gordon showcased his skills with 10 field '
 'goals made, proving to be a valuable asset for his team. Nikola Vucevic also '
 'stood out, making an impressive 12 field goals and securing 11 rebounds. '
 "Their exceptional performances contributed significantly to their team's "
 'success.\n'
 '\n'
 'On the other hand, some players had a tough time on the court. Bogdan '
 'Bogdanovic struggled with his shooting, missing 7 field goals. Michael '
 'Carter-Williams also faced difficulties, missing 11 field goals and '
 'committing 6 turnovers. Despite their challenges, they continued to fight '
 'and contribute in other areas of the game.\n'
 '\n'
 "These players' performances added an extra layer of excitement to the match. "
 "It's fascinating to witness the highs and lows of these talented athlet